# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

Der Profifußball ist nicht nur eine der weltweit populärsten Sportarten, sondern auch ein komplexes globales Wirtschaftsphänomen. Spielertransfers und die damit verbundenen Marktwerte von Profifußballern stellen zentrale Elemente dar, die sowohl für Vereine als auch für Investoren von entscheidender Bedeutung sind. Die Frage, wie der Marktwert eines Spielers bestimmt wird und welche Faktoren darauf einwirken, ist sowohl für die Sportökonomie als auch für datenwissenschaftliche Forschungen von großem Interesse.
Die Bewertung von Spielerleistungen und das Ermitteln ihrer finanziellen Wertigkeit ist jedoch keine triviale Aufgabe. Sie umfasst zahlreiche Variablen, darunter individuelle Leistungsdaten, Alter, Position, Verletzungshistorie, internationale Erfahrung sowie die Dynamik des Transfermarktes. 
Das Ziel dieses Projekts ist es, die zentralen Faktoren zu identifizieren, die den Marktwert eines Spielers beeinflussen, und ein Modell zu entwickeln, das auf Basis dieser Erkenntnisse Marktwerte mit hoher Genauigkeit vorhersagen kann. Hierbei kommen historische Daten, statistische Analysen und moderne Verfahren des maschinellen Lernens zum Einsatz

Wissenschaftliche Relevanz

Die Erforschung von Marktwertvorhersagen im Profifußball verbindet wissenschaftliche Disziplinen wie Statistik, Ökonomie, Informatik und Sportwissenschaft. Neben der Verbesserung von Entscheidungsprozessen für Vereine und Scouts bietet diese Arbeit auch wertvolle Erkenntnisse über die Mechanismen und Dynamiken eines der lukrativsten Sportmärkte der Welt.

Hypothese: Der Marktwert eines Profifußballspielers kann mit hoher Genauigkeit vorhergesagt werden, wenn Variablen wie individuelle Leistungsdaten, physische Eigenschaften, Alter, Markttrends und Transferhistorien systematisch analysiert und in ein maschinelles Lernmodell integriert werden.


### Setup

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
#import vegafusion as vf

In [2]:
pd.set_option('display.max_columns', None) 

### Import data

In [3]:
appearances = pd.read_parquet("../data/raw/appearances.parquet")
player_valuations = pd.read_parquet("../data/raw/player_valuations.parquet")
transfers = pd.read_parquet("../data/raw/transfers.parquet")
players = pd.read_parquet("../data/raw/players.parquet")

### Data structure

In [4]:
print("Appearances columns: \n", appearances.columns)
print("\n Player Valuations columns: \n", player_valuations.columns)
print("\n Transfers columns: \n", transfers.columns)
print("\n Players columns: \n", players.columns)

#### Appearances DataFrame
 **Beschreibung:** Diese Tabelle enthält historische Daten über die Leistungen eines Spielers in einzelnen Spielen.
 **Wichtige Spalten:**
 - `player_id`: Eindeutige Spieler-ID.
 - `date`: Datum des Spiels.
 - `player_name`: Name des Spielers.
 - `competition_id`: ID des Wettbewerbs.
 - `yellow_cards`, `red_cards`: Anzahl gelber und roter Karten.
 - `goals`, `assists`: Tore und Vorlagen im Spiel.
 - `minutes_played`: Anzahl der gespielten Minuten.

In [5]:
appearances.head(2)

#### Player Valuations DataFrame
 **Beschreibung:** Diese Tabelle zeigt die Marktwertentwicklungen der Spieler über die Zeit.
 **Wichtige Spalten:**
 - `player_id`: Eindeutige Spieler-ID.
 - `date`: Datum der Marktwertermittlung.
 - `market_value_in_eur`: Erster Marktwert in Euro.
 - `current_club_id`: ID des aktuellen Clubs.
 - `player_club_domestic_competition_id`: ID des nationalen Wettbewerbs des aktuellen Clubs.


In [6]:
player_valuations.head(2)

#### Transfers DataFrame
 **Beschreibung:** Diese Tabelle enthält Informationen zu Spielertransfers zwischen Clubs.
 **Wichtige Spalten:**
 - `player_id`: Eindeutige Spieler-ID.
 - `transfer_date`: Datum des Transfers.
 - `transfer_season`: Saison des Transfers.
 - `from_club_id`, `to_club_id`: IDs des abgebenden und aufnehmenden Clubs.
 - `from_club_name`, `to_club_name`: Namen der Clubs.
 - `transfer_fee`: Transfergebühr in Euro.
 - `market_value_in_eur`: Marktwert des Spielers zum Zeitpunkt des Transfers.

In [7]:
transfers.head(2)

#### Players DataFrame
 **Beschreibung:** Diese Tabelle enthält Basisinformationen zu Spielern.
 **Wichtige Spalten:**
 - `player_id`: Eindeutige Spieler-ID.
 - `name`, `first_name`, `last_name`: Name des Spielers.
 - `country_of_birth`, `country_of_citizenship`: Geburtsland und Staatsangehörigkeit.
 - `foot`: Bevorzugter Fuß (rechts/links).
 - `height_in_cm`: Körpergröße in cm.
 - `contract_expiration_date`: Vertragslaufzeitende.
 - `current_club_name`: Name des aktuellen Clubs.
 - `market_value_in_eur`: Aktueller Marktwert in Euro.
 - `highest_market_value_in_eur`: Höchster Marktwert in Euro.

In [8]:
players.head(2)

### Data corrections

#### Aggregierte Appearances DataFrame
- **Beschreibung:** Diese Tabelle fasst die Leistungen eines Spielers über alle Spiele zusammen.
- **Aggregierte Spalten:**
  - `total_minutes_played`: Gesamte Spielminuten.
  - `total_goals`, `total_assists`: Gesamte Tore und Vorlagen.
  - `total_yellow_cards`, `total_red_cards`: Summe der gelben und roten Karten.
  - `total_golas_per_game`: Totaler Durchschnitt der Tore pro Spiel 


In [9]:
appearances_agg = appearances.groupby('player_id').agg({
    'minutes_played': 'sum',  # Gesamtminuten
    'goals': 'sum',           # Gesamttore
    'assists': 'sum',         # Gesamtvorlagen
    'yellow_cards': 'sum',    # Gelbe Karten
    'red_cards': 'sum',       # Rote Karten
    'game_id': 'count'        # Anzahl der Spiele
}).reset_index()

appearances_agg.rename(columns={
    'minutes_played': 'total_minutes_played',
    'goals': 'total_goals',
    'assists': 'total_assists',
    'yellow_cards': 'total_yellow_cards',
    'red_cards': 'total_red_cards',
    'game_id': 'total_matches'
}, inplace=True)


appearances_agg.head()

#### Aggregierte Appearancespro Jahr (historisiert)
- **Beschreibung:** Diese Tabelle fasst die Leistung jährlich eines Spielers über alle Spiele zusammen.
- **Aggregierte Spalten:**
  - `minutes_played_per_game`: Gesamte Spielminuten.
  - `assists_per_game`: Gesamte Vorlagen.
  - `yellow_cards_per_game`, `red_cards_per_game`: Summe der gelben und roten Karten.
  - `golas_per_game`: Totaler Durchschnitt der Tore pro Spiel 


In [10]:
appearances['date'] = pd.to_datetime(appearances['date'])
appearances['year'] = appearances['date'].dt.year

appearances_hist = appearances.groupby(['player_id', 'year']).agg({
    'minutes_played': 'sum',
    'goals': 'sum',         
    'assists': 'sum',       
    'yellow_cards': 'sum',  
    'red_cards': 'sum',     
    'game_id': 'count' 
}).reset_index()

appearances_hist['goals_per_game'] = appearances_hist['goals'] / appearances_hist['game_id']
appearances_hist['assists_per_game'] = appearances_hist['assists'] / appearances_hist['game_id']
appearances_hist['yellow_cards_per_game'] = appearances_hist['yellow_cards'] / appearances_hist['game_id']
appearances_hist['red_cards_per_game'] = appearances_hist['red_cards'] / appearances_hist['game_id']
appearances_hist['minutes_played_per_game'] = appearances_hist['minutes_played'] / appearances_hist['game_id']

appearances_hist.drop(["minutes_played", "goals", "assists", "yellow_cards", "red_cards", "game_id"], axis=1, inplace =True)

appearances_hist.head()

In diesem Schritt werden die Leistungsdaten der Spieler, wie Tore, Assists und Karten, auf Spielebene normalisiert, um Metriken wie `goals_per_game` oder `assists_per_game` zu berechnen. Dadurch wird die Leistung unabhängig von der Anzahl absolvierter Spiele dargestellt, was eine objektivere Analyse ermöglicht und zeitliche Entwicklungen präziser abbildet. Dies erlaubt eine differenziertere Analyse, die weniger von der Anzahl der absolvierten Spiele beeinflusst ist.

player_data = appearances_hist[appearances_hist['player_id'] == 342229]
fig = px.line(
    player_data,
    x='year',
    y='goals_per_game',
    title='Tore pro Spiel im Zeitverlauf',
    labels={'goals_per_game': 'Tore pro Spiel', 'year': 'Jahr'}
)
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=min(player_data['year']),
        dtick=1  # Schritte von 1 Jahr
    )
)
fig.show()

#### Fehldende Werte raussuchen und Daten Engineering

In [11]:
print("Fehlende Werte in appearances:")
print(appearances.isnull().sum())

In [12]:
print("Fehlende Werte in player_valuations:")
player_valuations['date'] = pd.to_datetime(player_valuations['date'])
player_valuations['year'] = player_valuations['date'].dt.year

#player_valuations.drop("date", axis=1, inplace=True) # date wird gelöscht um auf Jahreseben den Marktwert zu untersuchen
player_valuations = player_valuations.drop_duplicates(subset=['player_id','year'], keep='last') # damit wird gewährleistet, dass pro Jahr auf nur ein Marktwert pro Spieler limitiert wird
print(player_valuations.isnull().sum())

Durch `drop_duplicates(subset=['player_id','year'], keep='last')` wird gewährleistet, dass es auf nur ein Marktwert pro Spieler im Jahr limitiert wird. Da es Spieler gibt deren Marktwert mehrmals im Jahr abgefragt wird.

In [13]:
print("Fehlende Werte in transfers:")
transfers['transfer_date'] = pd.to_datetime(transfers['transfer_date'])
transfers['year'] = transfers['transfer_date'].dt.year
# wichtig für das Mergen später
print(transfers.isnull().sum())

In [14]:
print("Fehlende Werte in players:")

print(players.isnull().sum())

In [15]:
players.dropna(subset=['market_value_in_eur'], inplace=True)

In [16]:
print("Fehlende Werte in appearances_hist:")
print(appearances_hist.isnull().sum())

#### Merging Datasets

`combined_data_agg` enthält die aggregierte Version (`appearances_agg`) der Spieler. Keine Historie enthalten durch `drop_duplicates(subset='player_id', keep='last')` wird der aktuellste Marktewert genommen.

combined_data_agg = player_valuations.drop_duplicates(subset='player_id', keep='last').merge(players, on='player_id', how='inner')
combined_data_agg = combined_data_agg.merge(appearances_agg, on='player_id', how='inner')

combined_data_agg = combined_data_agg.merge(transfers[['player_id', 'transfer_fee', 'market_value_in_eur','year']], 
                                    on=['player_id', 'year'], how='left')

combined_data_agg.head()


Agg geht auch so, vlt sogar besser ohne valuations (die hist) sondern nur nur ein Eintrag pro Spieler, ohne Datum das heißt auch ohne Transfer

In [17]:
combined_data_agg = players.merge(appearances_agg, on='player_id', how='inner')

#combined_data_agg = combined_data_agg.merge(transfers[['player_id', 'transfer_fee', 'market_value_in_eur','year']].drop_duplicates(subset='player_id', keep='first'), 
#                                    on=['player_id'], how='left')

combined_data_agg.head()

In [18]:
combined_data_agg[combined_data_agg["name"]=="Kylian Mbappé"]

`combined_data_hist` enthält die historisierte Version (`appearances_hist`) der Aggregationen 

In [19]:
combined_data_hist = player_valuations.merge(players, on='player_id', how='inner')
combined_data_hist = combined_data_hist.merge(appearances_hist, on=['player_id', 'year'], how='inner')

combined_data_hist = combined_data_hist.merge(transfers[['player_id', 'transfer_fee', 'market_value_in_eur', 'year', 'transfer_season']], 
                                    on=['player_id', 'year'] , how='left')

combined_data_hist.head()

`combined_data` enthält jeweils beide Aggregation Typen

combined_data = combined_data_hist.merge(appearances_agg, on='player_id', how='inner')
combined_data.head()

#### Feature Engineering

In [20]:
combined_data_hist.rename(columns={
    'market_value_in_eur_x': 'historical_to_date__market_value',
    'market_value_in_eur_y': 'current_market_value',
    'market_value_in_eur': 'transfer_market_value',
    'player_club_domestic_competition_id': 'league_id'
}, inplace=True)

combined_data_agg.rename(columns={
    'market_value_in_eur': 'current_market_value',
    'current_club_domestic_competition_id': 'league_id'
}, inplace=True)

In [21]:
## Feautre Engineering: transfer gap
combined_data_hist['transfer_fee_gap'] = combined_data_hist['transfer_fee'] - combined_data_hist['transfer_market_value']
combined_data_hist.drop(columns=['current_club_id_x', 'current_club_id_y', 'image_url', 'url', 'agent_name', 'player_code', 'first_name', 'last_name', 'current_club_domestic_competition_id'], inplace=True)

In [22]:
combined_data_agg.drop(columns=['image_url', 'url', 'agent_name', 'player_code', 'first_name', 'last_name'], inplace=True)

`transfer_fee_gap` enthält die Differenz zwischen der Transfer Gebühr und den jeweiligen Marktwert im Jahr des Transfers. Da die Transfers jährlich vorliegen, werden der `transfer_fee_gap` nur bei der historischen Version hinzugefügt 

##### Alter ergänzen und evaluieren

In [23]:
## Feautre Engineering: Alter
combined_data_hist['age'] = (pd.to_datetime(combined_data_hist['date']).dt.year - 
                         pd.to_datetime(combined_data_hist['date_of_birth']).dt.year)

current_year = datetime.now().year
combined_data_agg['age'] = current_year - pd.to_datetime(combined_data_agg['date_of_birth']).dt.year


In [24]:
combined_data_hist[['date_of_birth', 'age']].head()

In [25]:
combined_data_agg.head(2)

In [26]:
combined_data_hist.head(2)

In [27]:
combined_data_hist.shape

In [28]:
combined_data_hist = combined_data_hist[combined_data_hist['age'] > 15]
combined_data_agg = combined_data_agg[combined_data_agg['age'] > 15]

combined_data_hist.shape

In diesem Schritt wird das Alter der Spieler zum Zeitpunkt der Marktwertbewertung berechnet, indem das Datum der Bewertung (`date`) und das Geburtsdatum des Spielers (`date_of_birth`) herangezogen werden. Anschließend wird der Datensatz gefiltert, sodass nur Spieler berücksichtigt werden, die zum Zeitpunkt der Bewertung ein Mindestalter von 15 Jahren erreicht haben.

Das Mindestalter von 15 Jahren wird als Untergrenze definiert, da Marktwertbewertungen für Spieler, die jünger als 15 Jahre alt sind, in der Regel unplausibel und fehlerhaft erscheinen. Solche Datensätze können als Ausreißer betrachtet werden und die statistische Analyse sowie die Vorhersagemodelle erheblich verfälschen.

##### Aktive oder in Rente 

In [29]:
current_year = 2025

combined_data_hist['is_retired'] = combined_data_hist['last_season'] <= (current_year - 5)

combined_data_agg['is_retired'] = combined_data_agg['last_season'] <= (current_year - 5)

Die neue  Variable `is_retired` erlaubt es, aktive und inaktive Spieler getrennt zu betrachten, was Verzerrungen in den Modellen reduzieren kann.

#### Change Datatypes

In [30]:
combined_data_hist.info()

In [31]:
# Alle IDs als Strings umwandeln
id_columns = [
    "player_id", "league_id"
]
combined_data_hist[id_columns] = combined_data_hist[id_columns].astype(str)

# Datumsangaben umwandeln
date_columns = ["date", "date_of_birth", "contract_expiration_date"]
for col in date_columns:
    combined_data_hist[col] = pd.to_datetime(combined_data_hist[col], errors='coerce')


print(combined_data_hist.info())


In [32]:
combined_data_agg.info()

In [33]:
# Alle IDs als Strings umwandeln
id_columns = [
    "player_id", "current_club_id", "league_id"
]
combined_data_agg[id_columns] = combined_data_agg[id_columns].astype(str)

# Datumsangaben umwandeln
date_columns = [ "date_of_birth", "contract_expiration_date"]
for col in date_columns:
    combined_data_agg[col] = pd.to_datetime(combined_data_agg[col], errors='coerce')


print(combined_data_agg.info())

In [35]:

# abspeichern in data/processed
combined_data_hist.to_parquet('../data/processed/combined_data_hist.parquet')
combined_data_agg.to_parquet('../data/processed/combined_data_agg.parquet')

### Analysis

Für die Analysen wird die aggregierte Version der Dataframes `combined_data_agg` genutzt, um das mehrfache Vorkommen von Spielern zu verhindern. 

In [34]:
fig = px.box(
    combined_data_hist, 
    y="current_market_value", 
    color=combined_data_hist['is_retired'].map({True: 'Retired', False: 'Active'}),
    title="Boxplot of Current Market Value, Divided by Active or Retired (2020) Player",
    labels={"current_market_value": "Current Market Value (in M€)"},
    hover_data=["name", "age"] 
)

fig.update_layout(
    yaxis=dict(title="Current Market Value (in M€)", showgrid=True),
    xaxis=dict(showticklabels=False),
    title=dict(x=0.5)  # Titel zentrieren
)

fig.show()

Die Boxplots zeigen die Verteilung der aktuellen Marktwerte von Fußballspielern, getrennt nach aktiven und pensionierten Spielern. Die Marktwerte konzentrieren sich überwiegend im unteren Bereich, wobei einige wenige aktive Spieler als Ausreißer deutlich höhere Werte aufweisen. Pensionierte Spieler haben insgesamt niedrigere Marktwerte und kaum Ausreißer, was auf eine homogenere Verteilung hinweist. Der Plot verdeutlicht die starke Schiefe der Marktwerte und die größere Streuung bei aktiven Spielern im Vergleich zu pensionierten.

#### Ohne Ausreißer

In [35]:
q1 = combined_data_agg["current_market_value"].quantile(0.25)
q3 = combined_data_agg["current_market_value"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filtern der Daten ohne Ausreißer
combined_data_agg_no_outliers = combined_data_agg[
    (combined_data_agg["current_market_value"] >= lower_bound) & 
    (combined_data_agg["current_market_value"] <= upper_bound)
]

In [36]:
combined_data_agg_no_outliers.shape

In [37]:
fig = px.box(
    combined_data_agg_no_outliers, 
    y="current_market_value", 
    title="Boxplot of Current Market Value, Divided by Active or Retired (2020) Player, Without Outliers",
    labels={"current_market_value": "Current Market Value (in M€)"},
    hover_data=["name", "age"]
)

fig.show()

In [38]:
fig = px.box(
    combined_data_agg_no_outliers, 
    y="current_market_value", 
    color=combined_data_agg_no_outliers['is_retired'].map({True: 'Retired', False: 'Active'}),
    title="Boxplot of Current Market Value, Divided by Active or Retired (2020) Player, Without Outliers",
    labels={"current_market_value": "Current Market Value (in M€)"},
    hover_data=["name", "age"],
)

fig.show()

In [39]:
fig = px.box(
    combined_data_agg_no_outliers, 
    y="current_market_value", 
    color=combined_data_agg_no_outliers['is_retired'].map({True: 'Retired', False: 'Active'}),
    title="Boxplot of Current Market Value, Divided by Active or Retired (2020) Player, Without Outliers",
    labels={"current_market_value": "Current Market Value (in M€)"},
    hover_data=["name", "age"],
)

fig.show()

#### Descriptive statistics

In [40]:
#combined_data.describe().T

numeric_combined_data = combined_data_agg.select_dtypes(include=["number"])
numeric_combined_data.describe().T

Die deskriptive Analyse zeigt eine große Vielfalt unter den Fußballspielern, sowohl in Bezug auf ihre Leistung als auch auf ihren Marktwert. Der durchschnittliche aktuelle Marktwert (`current_market_value`) liegt bei etwa 1,95 Millionen Euro, wobei die Werte von nur 10.000 Euro bis hin zu beeindruckenden 200 Millionen Euro reichen. Spieler haben im Durchschnitt 6,33 Tore (`total_goals`) und 4,99 Assists (`total_assists`) erzielt, mit maximalen Werten von 490 Toren und 224 Assists, was die Präsenz einiger außergewöhnlicher Akteure unterstreicht. Die durchschnittliche Anzahl absolvierter Spiele (`total_matches`) liegt bei rund 66, wobei einige Spieler bis zu 589 Spiele bestritten haben. Die durchschnittliche Spielzeit (`total_minutes_played`) beläuft sich auf 4.565 Minuten, mit einem Maximum von nahezu 50.000 Minuten. Disziplinarisch erhalten Spieler im Schnitt etwa 9,76 gelbe Karten (`total_yellow_cards`) und 0,25 rote Karten (`total_red_cards`) im Verlauf ihrer Karriere. Das durchschnittliche Alter der Spieler (`age`) beträgt 31 Jahre, mit einer Spannweite von 17 bis 55 Jahren, was sowohl die jungen Talente als auch erfahrene Veteranen repräsentiert.

combined_data_agg[combined_data_agg["age"]==55]

#### Exploratory data analysis

Für EDA lieber historisierte Daten nutzen? Spieler kommen öfters vor durch historisierung

In [41]:
sns.histplot(combined_data_agg['current_market_value'], bins=50, kde=True)
plt.title("Distribution of Current Market Values")
plt.show()

fig = px.scatter(combined_data_agg, x='age', y='current_market_value', 
                  title="Scatterplot: Current Market Value vs Age",
                  labels={'age': 'Age (in years)', 'current_market_value': 'Current Market Value (€)'}, 
                  hover_data=["name", "age"])
fig.show()

auch bei agg gibt es welche die öfters vorkommen hier versuchen den neusten Datensatz zu nehmen

In [42]:
combined_data_agg["league_id"]

In [43]:
fig = px.scatter(combined_data_agg, x='total_goals', y='current_market_value', 
                  title="Scatterplot: Current Market Value vs Total Goals",
                  labels={'total_goals': 'Total Goals', 'current_market_value': 'Current Market Value (€)'},
                  hover_data=["name"])

fig.show()

idee eine Person als Bsp mit Hist 

In [44]:
fig = px.scatter(combined_data_agg, x='total_assists', y='current_market_value', 
                  title="Scatterplot: Current Market Value vs Assist Per Game",
                  labels={'total_assists': 'Total Assists', 'current_market_value': 'Current Market Value (€)'},
                  hover_data=["name", "player_id"])

fig.show()

In [45]:
lineplot_data = combined_data_hist[combined_data_hist['player_id'].isin(["418560", "371998", "342229"])]

fig = px.line(
    lineplot_data,
    x='year',
    y='historical_to_date__market_value',
    color='name',  # Unterschiedliche Spieler durch Farben darstellen
    line_group='player_id',
    hover_data={
        'name': True,
        'goals_per_game': True,  # Tore als zusätzliche Info
        'assists_per_game': True,  # Assists
        'year': True,    # Jahr
    },
    title="Vergleich der Marktwertentwicklung mehrerer Spieler",
    labels={
        'name': "Name",
        'year': 'Jahr',
        'goals_per_game': 'Tore pro Spiel',
        'assists_per_game':'Assist pro Spiel',
        'historical_to_date__market_value': 'Marktwert (in €)'
    }
)

fig.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Marktwert (in €)",
    legend_title="Spieler",
    hovermode="x unified"
)

fig.show()


Das Diagramm zeigt die Marktwertentwicklung dreier Spieler – Kylian Mbappé, Erling Haaland und Vinícius Júnior – über die Jahre. Alle Spieler zeigen ein konstantes Wachstum ihres Marktwerts, was ihre steigende Leistung und Attraktivität auf dem Transfermarkt widerspiegelt. Besonders Kylian Mbappé sticht hervor: Ab 2017 steigt sein Marktwert rapide an und stabilisiert sich bei ca. 200 Millionen Euro, was seine Stellung als einer der teuersten Spieler der Welt unterstreicht. Im Gegensatz dazu zeigt Erling Haaland erst ab 2019 einen signifikanten Anstieg seines Marktwerts, der mit seinem internationalen Durchbruch korreliert. Vinícius Júnior weist eine stetige, aber weniger drastische Entwicklung auf, was auf eine kontinuierliche Leistungssteigerung hindeutet. Insgesamt spiegelt das Diagramm unterschiedliche Karriereverläufe wider, wobei bei Mbappé eine Stabilisierung und bei Haaland ein starkes Wachstum erkennbar ist.

In [46]:
fig = px.scatter(
    lineplot_data,
    x='year',
    y='historical_to_date__market_value',
    color='name',
    size='goals_per_game',  # Punktgröße basiert auf Toren pro Spiel
    hover_data={
        'name': True,
        'goals_per_game': True,
        'assists_per_game': True,
        'height_in_cm': True,
        'year': True,
        'age': True,
    },
    title="Vergleich der Marktwertentwicklung mit Toren",
    labels={
        'name': "Name",
        'year': 'Jahr',
        'historical_to_date__market_value': 'Marktwert (in €)',
        'goals_per_game': 'Tore pro Spiel',
        'assists_per_game':'Assist pro Spiel',
        'age': "Alter",
        'height_in_cm': "Größe in cm"
    }
)

fig.update_traces(mode='lines+markers')  # Linien und Marker
fig.update_layout(
    legend_title="Spieler",
    hovermode="x unified",
)

fig.show()


Diese Diagramm erweitert die Analyse, indem es den Marktwert mit der Anzahl der Tore pro Spiel (dargestellt durch die Kreisgrößen) verknüpft. Hier zeigt sich ein klarer Zusammenhang: Spieler mit höheren Torzahlen, wie Erling Haaland ab 2020, tendieren zu einem stark ansteigenden Marktwert. Kylian Mbappé hingegen bleibt unabhängig von seiner Torleistung auf konstant hohem Marktwert, was zeigt, dass sein Status als Starspieler und sein Werbewert eine zentrale Rolle spielen. Vinícius Júnior hat kleinere Kreise (weniger Tore pro Spiel), bleibt aber dennoch stabil im Marktwert. Besonders auffällig ist die Entwicklung Haalands: Seine Leistungssteigerung in Form vieler Tore führt zu einer entsprechenden Steigerung seines Marktwerts. Dieses Diagramm unterstreicht, wie sportliche Leistungen, insbesondere Tore, bei aufstrebenden Spielern den Marktwert beeinflussen, während bei etablierten Stars wie Mbappé andere Faktoren dominieren.

position in abhängikeit anderer
filtern auf postion und dann eda

Lables gleich machen

noch ein Dataframe für die besten fünf ligen ?

##### Analyse nach Postition

In [47]:
avg_value = combined_data_agg.groupby("position")["current_market_value"].mean().reset_index()
avg_value = avg_value.sort_values("current_market_value", ascending=False)
fig = px.bar(avg_value, x="position", y="current_market_value",
             title="Durchschnittlicher Marktwert pro Position",
             labels={'position': 'Position', 'current_market_value': 'Marktwert (€)'},
             color="current_market_value",  
             color_continuous_scale="magma" )
fig.update_coloraxes(colorbar_title="Marktwert (€)")
fig.show()


In [48]:
fig = px.scatter(combined_data_agg, x="height_in_cm", y="current_market_value", color="position",
                 hover_data=["name"], 
                 labels={'height_in_cm': 'Größe (in cm)', 'current_market_value': 'Marktwert (€)', 'name': "Name", "position": "Position"},
                 title="Alter vs. Marktwert nach Position")
fig.show()


In [49]:
fig = px.scatter(combined_data_agg, x="total_goals", y="current_market_value", color="position",
                 hover_data=["name"], 
                 labels={'total_goals': 'Gesamt Tore', 'current_market_value': 'Marktwert (€)', 'name': "Name", "position": "Position"},
                 title="Alter vs. Marktwert nach Position")
fig.show()

In [50]:
fig = px.scatter(combined_data_agg, x="total_goals", y="current_market_value", 
                 size="total_assists", color="position", hover_name="name",
                 labels={'total_goals': 'Gesamt Tore',"total_assists":"Gesamt Assists", 'current_market_value': 'Marktwert (€)', 'name': "Name", "position": "Position"},
                 title="Marktwert vs. Tore (Größe: Assists)")
fig.show()


In [51]:
fig = px.scatter(combined_data_agg, x='total_minutes_played', y='current_market_value', 
                  title="Scatterplot: Current Market Value vs Minutes Played",
                  labels={'minutes_played': 'Gespielte Minuten', 'current_market_value': 'Marktwert (€)'},
                  hover_data=["name"])

fig.show()

In [52]:
combined_data_hist

In [54]:

fig_data = combined_data_hist.groupby(['year', 'position'])['historical_to_date__market_value'].mean().reset_index()


fig = px.line(fig_data, x='year', y='historical_to_date__market_value', color='position',
              title='Entwicklung des durchschnittlichen Marktwerts nach Position',
              labels={'year': 'Datum', 
                      'historical_to_date__market_value': 'Durchschnittlicher Marktwert',
                      'position': 'Position'})


fig.update_layout(legend_title_text='Position')
fig.update_xaxes(title_text='Datum')
fig.update_yaxes(title_text='Durchschnittlicher Marktwert (€)')
fig.update_layout(
    hovermode="x unified"
)

fig.show()

In [53]:
goalkeepers = combined_data_hist[combined_data_hist['position'] == 'Goalkeeper']

fig = px.scatter(goalkeepers, 
                 x='height_in_cm', 
                 y='current_market_value',
                 hover_data=['name', 'age'],
                 labels={'current_market_value': 'Marktwert (€)', "age":"Alter", 'name': "Name", "position": "Position", 'height_in_cm': 'Größe (in cm)'},
                 title='Torwart: Größe vs. Marktwert')

fig.update_layout(
    xaxis_title='Größe (cm)',
    yaxis_title='Marktwert (€)'
)

fig.show()

In [55]:
defenders = combined_data_hist[combined_data_hist['sub_position'].isin(['Centre-Back', 'Left-Back', 'Right-Back'])]

fig = px.scatter(defenders, 
                 x='height_in_cm', 
                 y='current_market_value',
                 color='sub_position',
                 hover_data=['name', 'age', 'yellow_cards_per_game'],
                 labels={'current_market_value': 'Marktwert (€)',"yellow_cards_per_game":"Gelbe Karten Pro Spiel", "sub_position":"Sub-Position", "age":"Alter", 'name': "Name", "position": "Position", 'height_in_cm': 'Größe (in cm)'},
                 title='Verteidiger: Größe vs. Marktwert')

fig.update_layout(
    xaxis_title='Größe (cm)',
    yaxis_title='Marktwert (€)',
    legend_title='Position'
)

fig.show()

In [56]:
midfielders = combined_data_agg[combined_data_agg['sub_position'].isin(['Defensive Midfield', 'Central Midfield', 'Attacking Midfield'])]

fig = px.scatter(midfielders, 
                 x='total_assists', 
                 y='current_market_value',
                 color='sub_position',
                 hover_data=['name', 'age', 'total_goals'],
                 labels={'current_market_value': 'Marktwert (€)',"total_goals":"Gesamte Tore", "sub_position":"Sub-Position", "age":"Alter", 'name': "Name", "total_assists": "Gesamte Assists", 'height_in_cm': 'Größe (in cm)'},
                 title='Mittelfeldspieler: Vorlagen pro Spiel vs. Marktwert')

fig.update_layout(
    xaxis_title='Vorlagen pro Spiel',
    yaxis_title='Marktwert (€)',
    legend_title='Position'
)

fig.show()

In [57]:
forwards = combined_data_agg[combined_data_agg['sub_position'].isin(['Centre-Forward', 'Second Striker', 'Left Winger', 'Right Winger'])]

fig = px.scatter(forwards, 
                 x='total_goals', 
                 y='current_market_value',
                 color='sub_position',
                 hover_data=['name', 'age', 'total_assists'],
                 labels={'current_market_value': 'Marktwert (€)',"total_goals":"Gesamte Tore", "sub_position":"Sub-Position", "age":"Alter", 'name': "Name", "total_assists": "Gesamte Assists", 'height_in_cm': 'Größe (in cm)'},
                 title='Stürmer: Tore pro Spiel vs. Marktwert')

fig.update_layout(
    xaxis_title='Tore pro Spiel',
    yaxis_title='Marktwert (€)',
    legend_title='Position'
)

fig.show()

In [58]:
fig = px.box(combined_data_agg, x="foot", y="current_market_value", color="position",
             title="Marktwert nach bevorzugtem Fuß und Position",
             labels={'current_market_value': 'Marktwert (€)',"foot":"Fuß", "position":"Position"})

fig.update_layout(
    xaxis_title="Bevorzugter Fuß",
    yaxis_title="Marktwert (€)",
    legend_title="Position"
)

fig.show()

In [59]:
fig = px.box(combined_data_agg_no_outliers, x="foot", y="current_market_value", color="position",
             title="Marktwert nach bevorzugtem Fuß und Position",
             labels={'current_market_value': 'Marktwert (€)',"foot":"Fuß", "position":"Position"})

fig.update_layout(
    xaxis_title="Bevorzugter Fuß",
    yaxis_title="Marktwert (€)",
    legend_title="Position"
)

fig.show()

In [60]:

top_5_leagues = ['GB1', 'ES1', 'FR1', 'IT1', 'L1']


filtered_data = combined_data_agg[combined_data_agg['league_id'].isin(top_5_leagues)]

fig = px.box(filtered_data, x="foot", y="current_market_value", color="position",
             facet_col="league_id",
             title="Marktwert nach bevorzugtem Fuß, Position und Liga",
             labels={'current_market_value': 'Marktwert (€)',"foot":"Fuß", "position":"Position", "league_id":"Liga"})

fig.update_layout(
    xaxis_title="Bevorzugter Fuß",
    yaxis_title="Marktwert (€)",
    legend_title="Position"
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()


In [61]:


top_5_leagues = ['GB1', 'ES1', 'FR1', 'IT1', 'L1']


active_players = combined_data_agg[(combined_data_agg["is_retired"] == False) & (combined_data_agg["league_id"].isin(top_5_leagues))]


top_countries = active_players.groupby("country_of_citizenship").agg({
    "current_market_value": "mean",
    "player_id": "count"
}).reset_index()

min_players = 50
top_countries_filtered = top_countries[top_countries["player_id"] >= min_players]

top_10_countries = top_countries_filtered.sort_values("current_market_value", ascending=False).head(10)

fig = px.bar(top_10_countries, 
             x="country_of_citizenship", 
             y="current_market_value",
             color="player_id",  # Farbkodierung basierend auf der Anzahl der Spieler
             text="player_id",
             labels={'current_market_value': 'Marktwert (€)',"foot":"Fuß", "country_of_citizenship":"Staatsbügerschaft", "league_id":"Liga"},
             title=f"Top 10 Länder nach durchschnittlichem Marktwert (nur aktive Spieler in Top 5 Ligen, mind. {min_players} Spieler)")

fig.update_layout(
    xaxis_title="Land der Staatsbürgerschaft",
    yaxis_title="Durchschnittlicher Marktwert (€)",
    xaxis_tickangle=-45,
    coloraxis_colorbar_title="Anzahl der Spieler"
)


fig.update_traces(texttemplate='%{text} Spieler', textposition='outside')

fig.show()



#### Relationships

Frage welche Bedeutung hat Multikorrelation hier

In [62]:
correlation_matrix = numeric_combined_data.corr()

plt.figure(figsize=(18, 16)) 
sns.set_theme(style="white", font_scale=1.1)

# Heatmap zeichnen
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt=".2f", 
    cmap="coolwarm", 
    cbar_kws={'shrink': 0.8},  
    linewidths=0.5, 
    square=True
)

# Titel und Anzeige
plt.title("Korrelation der Merkmale", fontsize=20, pad=30)
plt.xticks(rotation=45, ha="right")  
plt.yticks(rotation=0)  
plt.tight_layout()  
plt.show()

Die Korrelationsmatrix zeigt die linearen Zusammenhänge zwischen den verschiedenen numerischen Merkmalen des Datensatzes, die zur Vorhersage des Marktwertes von Fußballspielern verwendet werden. Werte nahe 1 oder -1 deuten auf starke positive bzw. negative Korrelationen hin, während Werte nahe 0 auf geringe oder keine Korrelation hinweisen. Beispielsweise zeigt die Variable "highest_market_value_in_eur" mit 0.72 eine starke positive Korrelation mit dem aktuellen Marktwert ("current_market_value"), was intuitiv Sinn ergibt, da der bisher höchste Marktwert eines Spielers ein guter Indikator für dessen aktuellen Marktwert ist. Variablen wie "age" und "red_cards" zeigen eher geringe oder negative Korrelationen, was darauf schließen lässt, dass diese Faktoren weniger stark mit dem Marktwert in Zusammenhang stehen.

In [63]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Features auswählen
features = numeric_combined_data

# Entfernen von fehlenden Werten
features = features.dropna()

# Konstanten hinzufügen
features_with_constant = sm.add_constant(features)

# VIF berechnen
vif_data = pd.DataFrame()
vif_data['Variable'] = features_with_constant.columns
vif_data['VIF'] = [variance_inflation_factor(features_with_constant.values, i) for i in range(features_with_constant.shape[1])]

print(vif_data)


anova

In [67]:
# Kopiere die Daten für die Bereinigung
data_cleaned = numeric_combined_data

# Entferne problematische oder redundante Variablen
data_cleaned = data_cleaned.drop(columns=[
    #'transfer_fee',              
    #'transfer_market_value',     
    #'transfer_fee_gap',          
    'total_minutes_played',      
    'total_goals',               
    'total_assists',             
    'total_yellow_cards',        
    'total_red_cards',           
    'total_matches'              
])

features = data_cleaned.dropna()

# Konstanten hinzufügen
features_with_constant = sm.add_constant(features)

# VIF berechnen
vif_data = pd.DataFrame()
vif_data['Variable'] = features_with_constant.columns
vif_data['VIF'] = [variance_inflation_factor(features_with_constant.values, i) for i in range(features_with_constant.shape[1])]

print(vif_data)

Der Variance Inflation Factor (VIF) misst die Multikollinearität zwischen unabhängigen Variablen in einem Datensatz. Ein hoher VIF-Wert weist darauf hin, dass eine Variable stark mit anderen korreliert ist, was in Modellen wie der linearen Regression zu Verzerrungen führen kann. In diesem Schritt wurden die VIF-Werte für alle Variablen berechnet, problematische Variablen mit sehr hohen VIF-Werten identifiziert und entweder entfernt oder so transformiert, dass die Multikollinearität reduziert wurde. Ziel war es, ein Dataframe mit unabhängigen Prädiktoren zu erstellen.

In [65]:
data_cleaned

In [ ]:
correlation_matrix = numeric_combined_data.corr()

plt.figure(figsize=(18, 16)) 
sns.set_theme(style="white", font_scale=1.1)


sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt=".2f", 
    cmap="coolwarm", 
    cbar_kws={'shrink': 0.8},  
    linewidths=0.5, 
    square=True
)


plt.title("Korrelation der Merkmale", fontsize=20, pad=30)
plt.xticks(rotation=45, ha="right")  
plt.yticks(rotation=0)  
plt.tight_layout()  
plt.show()

In [66]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Features auswählen
features = numeric_combined_data

# Entfernen von fehlenden Werten
features = features.dropna()

# Konstanten hinzufügen
features_with_constant = sm.add_constant(features)

# VIF berechnen
vif_data = pd.DataFrame()
vif_data['Variable'] = features_with_constant.columns
vif_data['VIF'] = [variance_inflation_factor(features_with_constant.values, i) for i in range(features_with_constant.shape[1])]

print(vif_data)

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

#### Variable lists

In [130]:
combined_data_filtered.columns

doppelte rausmachen

In [131]:
features = [
    "height_in_cm", "age", "minutes_played", "goals", "assists", 
    "yellow_cards", "red_cards", "transfer_fee", "transfer_fee_gap", 
    "country_of_citizenship", "foot", "last_season", "position", 'goals_per_game',
       'assists_per_game', 'yellow_cards_per_game', 'red_cards_per_game',
       'minutes_played_per_game','age', 'is_retired'
]

target = "current_market_value"

features = [
    "height_in_cm", "age", "minutes_played", "goals", "assists", 
    "yellow_cards", "red_cards", "transfer_fee", "transfer_fee_gap", 
    "country_of_citizenship", "foot"
]

### Select model

In [146]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
lr_model = LinearRegression()

### Modell mit Ausreißer

#### Data splitting

In [185]:
X = pd.get_dummies(combined_data_filtered[features], drop_first=True)
y = combined_data_filtered[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [186]:
print(X.shape)
print(y.shape)

#### Training and validation

##### Lineare Regression

In [187]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())

In [188]:
X_train_filled = X_train.fillna(X.median())
X_test_filled = X_test.fillna(X.median())

In [190]:
print(f"Länge von X_train_filled: {len(X_train_filled)}")
print(f"Länge von y_train: {len(y_train)}")

In [191]:
lr_model.fit(X_train_filled, y_train)

##### RandomForestRegressor

In [192]:
rf_model.fit(X_train, y_train)

### Modell ohne Ausreißer

#### Data splitting

In [163]:
X_no = pd.get_dummies(combined_data_no_outliers[features], drop_first=True)
y_no = combined_data_no_outliers[target]
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(X_no, y_no, test_size=0.2, random_state=42)

#### Training and validation

##### Lineare Regression

In [164]:
print(X_train_no.isnull().sum())
print(X_test_no.isnull().sum())

In [165]:
X_train_filled_no = X_train_no.fillna(X.median())
X_test_filled_no = X_test_no.fillna(X.median())

In [166]:
print(f"Länge von X__train_filled: {len(X_train_filled_no)}")
print(f"Länge von y_train: {len(y_train_no)}")

In [176]:
lr_model_no = lr_model.fit(X_train_filled_no, y_train_no)

##### RandomForestRegressor

In [173]:
rf_model_no = rf_model.fit(X_train_no, y_train_no)

### Fit model

In [193]:
lr_predictions = lr_model.predict(X_test_filled)
lr_predictions_no = lr_model_no.predict(X_test_filled_no)

In [194]:
rf_predictions = rf_model.predict(X_test)
rf_predictions_no = rf_model_no.predict(X_test_no)

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


### Evaluation on test set

In [195]:
lr_mse = mean_squared_error(y_test, lr_predictions)
lr_r2 = r2_score(y_test, lr_predictions)
print("Linear Regression - Mean Squared Error:", lr_mse)
print("Linear Regression - R-squared:", lr_r2)

lr_mse_no = mean_squared_error(y_test_no, lr_predictions_no)
lr_r2_no = r2_score(y_test_no, lr_predictions_no)
print("Linear Regression No Outliers - Mean Squared Error:", lr_mse_no)
print("Linear Regression No Outliers - R-squared:", lr_r2_no)

In [196]:
# Evaluating the models using Mean Squared Error and R-squared.
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)
print("Random Forest - Mean Squared Error:", rf_mse)
print("Random Forest - R-squared:", rf_r2)

rf_mse_no = mean_squared_error(y_test_no, rf_predictions_no)
rf_r2_no = r2_score(y_test_no, rf_predictions_no)
print("Random Forest No Outliers - Mean Squared Error:", rf_mse_no)
print("Random Forest No Outliers - R-squared:", rf_r2_no)

Warum ist das Ergebnis so schlecht ohne Ausreißer

Der hohe MSE resultiert vor allem aus der Skalierung der Zielvariable, den Marktwerten von Fußballspielern, die teilweise extrem hoch ausfallen. Da der MSE die Fehlerwerte quadriert, wirken sich auch kleinere Abweichungen bei hohen Marktwerten überproportional stark auf das Gesamtergebnis aus. Zusätzlich tragen die große Varianz in den Daten und das Vorhandensein von Outliern – also Spielern mit außergewöhnlich hohen Marktwerten – dazu bei, dass die Fehlerspanne weiter ansteigt. Diese extremen Werte sind schwer vorherzusagen und führen zu einer Verschlechterung des MSE.

In [ ]:
residuals = y_test - lr_predictions

plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='k', alpha=0.7)
plt.title("Verteilung der Residuen (Linear Regression)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

In [197]:
residuals = y_test - lr_predictions

plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='k', alpha=0.7)
plt.title("Verteilung der Residuen (Linear Regression)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

Die Verteilung der Residuen zeigt, wie genau ein Modell die tatsächlichen Werte vorhersagen kann. Bei der Linearen Regression ist die Fehlerverteilung zwar symmetrisch um den Wert Null, jedoch deutlich breiter. Dies bedeutet, dass die Lineare Regression größere Fehler (Residuen) produziert. Insbesondere zeigt der rechte Rand der Verteilung, dass das Modell einige Vorhersagen erheblich überschätzt hat. Die Häufigkeit von Residuen nahe Null ist zwar hoch, dennoch führen die großen Fehler dazu, dass die Leistung des Modells beeinträchtigt wird. Diese breitere Streuung deutet darauf hin, dass die Lineare Regression die nichtlinearen und komplexen Zusammenhänge in den Daten nicht ausreichend abbilden kann.

In [ ]:
residuals = y_test - rf_predictions

plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='k', alpha=0.7)
plt.title("Verteilung der Residuen (Random Forest Regression)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

In [198]:
residuals = y_test - rf_predictions

plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='k', alpha=0.7)
plt.title("Verteilung der Residuen (Random Forest Regression)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

In [199]:
residuals = y_test_no - rf_predictions_no

plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='k', alpha=0.7)
plt.title("Verteilung der Residuen (Random Forest Regression)")
plt.xlabel("Residuum")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.show()

Im Vergleich dazu zeigt die Random Forest Regression eine wesentlich engere Fehlerverteilung, bei der die meisten Residuen deutlich näher an Null liegen. Dies verdeutlicht, dass die Vorhersagen des Random Forest Modells genauer sind und der Marktwert der Spieler besser approximiert wurde. Die Random Forest Regression ist ein nichtlineares Modell, das in der Lage ist, komplexe Muster und Beziehungen in den Daten besser zu erfassen. Dadurch reduziert es die Anzahl großer Fehler und zeigt eine insgesamt konzentriertere Residuenverteilung mit weniger Ausreißern.

## Discussion + Conclusion


Emotionen

> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.